# **NLP Project: Analyzing and Predicting Salary from Job Description**

*Master in Machine Learning for Health, 2023~2024*

*Authors: Daniel Corrales, Jaime Fernández & Rafael Rodríguez*

---

In [ ]:
# Common imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import subprocess
import os

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#To wrap long text lines
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# For fancy table Display
%load_ext google.colab.data_table

In [ ]:
import spacy
import gensim
print(spacy.__version__)

In [ ]:
# Download spaCy model
!python -m spacy download en_core_web_md # Or other

## **0. Data Loading**

In [ ]:
# TODO

## **1. Preprocessing Pipeline**

Here we preprocess the corpus to obtain its lemmatized version and we perform N-gram detection.

In [ ]:
nlp = spacy.load('en_core_web_md')
nlp.disable('parser')
nlp.disable('ner')

# Assume that docs are in corpus list
# Corpus list contains all documents (train, val and test)
lemmatized_corpus = [[tk.lemma_ for tk in nlp(doc.lower()) if (tk.is_alpha or tk.is_digit) \
                      and not tk.is_stop and not tk.is_punct] for doc in corpus]

print(f"Number of documents: {len(lemmatized_corpus)}")

### N-gram Detection

In [ ]:
# Firts step is to find N-grams to improve LDA's performance
n_gram_model = gensim.models.phrases.Phrases(lemmatized_corpus, min_count=2, threshold=20)
n_gram_corpus = [el for el in n_gram_model[lemmatized_corpues]]

# Display table with N-grams
n_gram_dict = {}
detected_n_grams = []

for doc in n_gram_corpus:
  for word in doc:
    if '_' in word:
      if word not in detected_n_grams:
        detected_n_grams.append(word)
        n_gram_dict[word] = 0

      n_gram_dict[word] += 1

n_grams_df = pd.DataFrame([(key, value) for key, value in n_gram_dict.items()], columns=['N_gram', 'Count']).sort_values(by='Count', ascending=False)
n_grams_df

## **2. Vectorization**

Three main vectorizations are generated here: BoW, TF-IDF and word embeddings.

In [ ]:
D = gensim.corpora.Dictionary(n_gram_corpus)
len_bf = len(D)

D.filter_exptremes(no_below=4, no_above=0.8, keep_n=5000)
len_af = len(D)

print(f"Dictionary length before filtering: {len_bf}")
print(f"Dictionary length after filtering: {len_af}")

### Bag of Words

In [ ]:
corpus_bow_sparse = [D.doc2bow(doc) for doc in n_gram_corpus]
corpus_bow_dense = gensim.matutils.corpus2dense(corpus_bow_sparse, num_terms=len(D))
print(corpus_bow_dense.shape)

### TF-IDF

In [ ]:
tfidf = gensim.models.TfidfModel(courpus_bow_sparse)
corpus_tfidf_sparse = tfidf[corpus_bow_sparse]
corpus_tfidf_dense = gensim.matutils.corpus2dense(corpus_tfidf_sparse, num_terms=len(D))

### Word Embeddings

In [ ]:
# TODO

## **3. Topic Modeling: Latent Dirichlet Allocation**

Latent Dirichlet Allocation is peformed to analyze the sematic structuture of the corpus in terms of topics and to obtain the LDA document vectorization.

### Latent Dirichlet Allocation with Mallet

In [ ]:
def install_java():
    !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    !java -version
install_java()

In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [ ]:
os.environ['MALLET_HOME'] = 'mallet-2.0.8'
mallet_path = 'mallet-2.0.8/bin/mallet'

Import corpus to *.txt file.

In [ ]:
with open('n_gram_corpus.txt', 'w'):
  for i, doc in n_gram_corpus:
    f.write(f"{i} 0 {" ".join(doc)}")
  f.close()

!mallet-2.0.8/bin/mallet import-file --input mycorpus.txt --output mycorpus.mallet --keep-sequence --remove-stopwords

Analyze coherence for different number of topics.

In [ ]:
def compute_mallet_LDA(n_topics, corpus, coherence=True):
  # Train LDA
  print(f"Mallet LDA n_topics {n_topics}")
  command = f"mallet-2.0.8/bin/mallet train-topics --input mycorpus.mallet --num-topics {n_topics} --num-iterations 1000 \
              --output-doc-topics doc_topics.txt --word-topic-counts-file wtc_counts.txt --topic-word-weights-file \
              topic_weights.txt --output-topic-keys topic_keys.txt --num-top-words 20"
  subprocess.call(command, shell=True)

  # Retrieve doc-topics matrix and sparsify
  thetas = []
  with open('doc_topics.txt', 'r') as f:
    doc_topic = f.readlines()
    f.close()

  for line in doc_topic:
    digits = line.split("\t")[2:]
    thetas.append(np.array([float(dig) for dig in digits]))

  thetas = np.vstack(thetas)
  thetas = np.where(thetas < 0.01, 0, thetas)
  thetas = [[(i, prob) for i, prob in enumerate(doc) if prob != 0.0] for doc in thetas]

  # Create corpus and dictionary in Mallet format
  with open("topic_keys.txt") as fin:
    tpc_descriptions = fin.readlines()
    tpc_descriptions = [el.strip().split("\t")[-1].split() for el in tpc_descriptions]

  Dict = gensim.corpora.Dictionary(corpus)

  # Compute coherence
  if coherence:
    nwords = 10
    cm = CoherenceModel(topics=tpc_descriptions, texts=corpus, dictionary=Dict, coherence='c_v', topn=n_words)
    coherence = np.mean(cm.get_coherence_per_topic())
    return thetas, tpc_descriptions, Dict, coherence

  return thetas, tpc_descriptions, Dict

In [ ]:
n_topics = [5, 10, 15, 20, 25, 50]
coherence_list_mallet = []

for n in n_topics:
  _, _, _, coherence = compute_mallet_LDA(n, corpus=n_gram_corpus, coherence=True)
  coherence_list_mallet.append(coherence)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(n_topics, coherence_list_mallet, label='Mallet', marker='o')
plt.xlabel('Number of topics')
plt.ylabel('Coherence')
plt.title('Mallet LDA coherence')
plt.grid()
plt.show()

Retrain the model with highest coherence.

In [ ]:
idx_max = np.argsort(coherence_list_mallet)[-1]
best_n_topics = n_topics[idx_max]

LDA_corpus, tpc_descriptions, Dict = compute_mallet_LDA(best_n_topics, corpus=n_gram_corpus, coherence=False) # LDA_corpus contains the vectorization

### Visualization and Analysis

In [ ]:
# TODO

## **4. Supervised Learning Task**

### Classical Models: ...

In [ ]:
# TODO

### Transformers

In [ ]:
# TODO

## **5. Semantic Graph and Distances between Documents**

Convert to sparse matrix representation.

In [ ]:
# Convert from list of tuples to sparse matrix
# TODO: LDA_corpus

LDA_sparse = corpus2csc(LDA_corpus).T
n_topics = LDA_sparse.shape[1]
n_docs = LDA_sparse.shape[0]

print(f"Number of topics: {n_topics}")
print(f"X: sparse matrix with {LDA_sparse.nnz} nonzero values out of {n_docs * n_topics}")
print(LDA_sparse.shape)

Renormalize rows to get probabilistic embeddings.

In [ ]:
print(f"Average row sum before normalizing: {np.mean(LDA_sparse.sum(axis=1).T)}")

LDA_sparse = scsp.csr_matrix(LDA_sparse / np.sum(LDA_sparse, axis=1))

print(f"Average row sum after normalizing: {np.mean(LDA_sparse.sum(axis=1).T)}")

Compute Similarity matrix based on Battacharyya Coefficient.

In [ ]:
S = np.sqrt(LDA_sparse) * np.sqrt(LDA_sparse.T)

print(f"Shape of S: {S.shape}")
print(f"Number of non-zero elements: {S.nnz}")
print(f"Proportion of non-zero values: {S.nnz / (S.shape[0] * S.shape[1]) * 100:.3f}%")

In [ ]:
S = scsp.triu(S, k=1) # Mantain upper triangular matrix
print('Number of non-zero components in S:', S.nnz)

Threshold similarity matrix to discard non important edges.

In [ ]:
n_nodes = LDA_sparse.shape[0]
n_edges = S.nnz
n_edges_per_node = n_edges / n_nodes

print(f"Number of nodes: {n_nodes}")
print(f"Number of edges: {n_edges}")
print(f"Number of edges per node: {n_edges_per_node}")

In [ ]:
n_edges_per_node = 10

n_edges = n_edges_per_node * n_nodes
sorted_sim = np.sort(S.data)[::-1]
thr = sorted_sim[n_edges]

print(f"Threshold: {thr}")
print(f"Number of edges: {n_edges}")

plt.figure(figsize=(8, 2))
plt.hist(S.data, bins=100)
plt.axvline(thr, 'r-')
plt.xlabel('Similarity values')
plt.ylabel('No. of matrix entries')
plt.show()

In [ ]:
S.data = np.where(S.data < thr, 0, S.data)
S.eliminate_zeros()

Plot graph and compute layout.

In [ ]:
G = nx.from_scipy_sparse_array(S)
positions = nx.spring_layout(G, iterations=50, seed=0)

plt.figure(figsize=(7,7))
nx.draw(G, positions, node_size=2, width=0.02)

Find connected components and plot them separately.

In [ ]:
nodes_lcc = list(nx.connected_components(G))[0]
n_conn_comp = len(nodes_lcc)

fig, axes = plt.subplots(n_conn_comp, 1, figsize=(15,15), layout='tight')

for i, conn_comp in enumerate(nodes_lcc):
  G_lcc = G.subgraph(conn_comp)
  positions_lcc = nx.spring_layout(G_lcc, iterations=50, seed=0)

  nx.draw(G_lcc, positions_lcc, node_size=2, width=0.06, ax=axes[i])
  axes[i].set_title(f"Connected Component {i+1}")

fig.show()

Community detection algorithm to explore semantic structure of corpus and similarities between documents.

In [ ]:
C = nx_comm.louvain_communities(G, seed=0)

# Metrics of the partition
modularity = nx_comm.modularity(G, C)
coverage, performance = nx_comm.partition_quality(G, C)

nc = len(C)
print(f"Number of communities: {nc}")
print(f"Modularity: {modularity}")
print(f"Coverage: {coverage}")
print(f"Performance: {performance}")

In [ ]:
palette = sns.color_palette(palette="Paired", n_colors=nc)

node2comm = {n: 0 for n in G}
for i, com in enumerate(C):
    for node in list(com):
        node2comm[node] = i

# Map node attribute to rgb colors
node_colors = [palette[node2comm[n]] for n in G]

# Get list of degrees
degrees = [val / 3 for (node, val) in G.degree()]

#  Draw graph
plt.figure(figsize=(7, 7))
nx.draw(G, positions, node_size=degrees, node_color=node_colors, width=0.1)

Analyze communities in terms of topics.

In [ ]:
# TODO: Personalize with our implementation

# col = 'main_topic_35'
# newcols=['Community', 'Main topic', 'Topic 2', 'Topic 3', 'Topic 4']

# df = pd.DataFrame(columns=newcols).set_index('Community')

# # For each community...
# for i, docs in enumerate(C):

#     # Join all keywords from all documents in the community
#     words = ",".join(NSF_df.iloc[list(docs)][col]).split(",")

#     # Count the number of occurences of each keyword
#     kwd_count = Counter(words)
#     n_votes = sum(kwd_count.values())

#     # Keep keywords that are frequent enough
#     kwds = [k for k, v in kwd_count.items() if v > 0.12 * n_votes]
#     kwds = sorted(kwds, key= lambda x: kwd_count[x], reverse=True)
#     kwds = [f'C{i}'] + kwds + [""] * (len(newcols) - len(kwds) -1)

#     row = pd.Series(dict(zip(newcols, kwds))).to_frame().T.set_index('Community')
#     df = pd.concat([df, row], axis=0)

# df

## **6. Extension Work**

In [ ]:
# TDOD